In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

path = 'C:/Users/queenSSH/Desktop/학교/CUAI/cuai하계/'

naver = pd.read_csv(path + '네이버_추천시스템.csv' , encoding='cp949')
daum = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')

naver = naver[['title', 'story']]
daum = daum[['title','story']]

#web_combined = pd.concat([naver, daum], axis=0, ignore_index=True)
web_combined = daum
#web_combined = naver

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np

class SentenceTokenizer(object):
    def __init__(self) -> None:
        self.kkma = Kkma()
        self.twitter = Twitter()
#         self.stopwords = ['중인', '만큼', '마찬가지', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가']
        self.stopwords = ['을', '를', '에', '의', '가','우리','저희','그녀']

#     def excel2sentences(self, file):
#         story = pd.read_csv(file,  encoding='cp949')
#         story = story['story']
#         #story.parse()
#         #sentences = self.kkma.sentences(story.text)
#         sentences = self.kkma.sentences(story)

#         for i in range(0, len(sentences)):
#             if len(sentences[i]) <= 10:
#                 sentences[i - 1] += (' ' + sentences[i])
#                 sentences[i] = ''

#         return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for i in range(0, len(sentences)):
            if len(sentences[i]) <= 10:
                sentences[i - 1] += (' ' + sentences[i])
                sentences[i] = ''

        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence)) if noun not in self.stopwords and len(noun) > 1]))

        return nouns



In [3]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]

        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            
            if link_sum != 0:
                A[:, id] /= link_sum
            
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def keywords(self, word_num=5):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


## Naver

In [4]:
web_combined = naver

In [5]:
key_wordlist = []
import warnings
warnings.filterwarnings('ignore')

for i in range(0, len(web_combined['story'])):
    try :
        
        text = web_combined['story'][i]
        textrank = TextRank(text)
        print('keywords :', textrank.keywords())
        key_wordlist.append(textrank.keywords())
        
    except:
        key_wordlist.append("none")
    

keywords : ['부대', '마물', '전담', '입대', '마법']
keywords : ['게임', '고백', '마스터', '소프트', '선배']
keywords : ['경주로', '대결', '운명', '자리', '인의']
keywords : ['사이']
keywords : ['권한', '모든', '암호', '절망', '캐리어']
keywords : ['하루하루', '학년', '고등학교', '모든', '목표']
keywords : ['성찬', '가족', '사건', '살인', '현장']
keywords : ['외면', '창궐', '극한', '미지', '바이러스']
keywords : ['뒷바라지', '목숨', '아들', '엄마']
keywords : ['마약', '사냥', '코미디', '로맨스', '액션']
keywords : ['아빠', '만난', '사람', '아버지', '노력']
keywords : ['사화', '인간', '신선', '백화', '소망']
keywords : ['정호', '자신', '아내', '별거', '숙취']
keywords : ['용수', '비디오', '모금', '사채', '업자']
keywords : ['교생', '구원', '살인마', '왕따']
keywords : ['순이', '고등학교', '과연', '관심', '마무리']
keywords : ['귀신', '마피아게임', '시즌', '잔혹', '초대']
keywords : ['당신', '손님', '조향사', '다시', '순간']
keywords : ['문제', '정말', '기분', '경험', '이성']
keywords : ['전우치', '진의', '신입', '장손', '전기']
keywords : ['건물', '누명', '약품', '영문', '타깃']
keywords : ['기억', '감방', '남자', '사실', '자신']
keywords : ['단편', '데드', '실험', '공포', '버스']
keywords : ['인생', '혼인', '금지', '사기', '상대로']
ke

keywords : ['마주', '연애', '다시', '모든', '바로']
keywords : ['꼰대', '셔틀', '이제', '조폭', '일진']
keywords : ['동우', '바리스타', '사기', '스무', '오반']
keywords : ['돌이', '동료', '물리', '악당', '동양']
keywords : ['고해', '남자', '사랑', '사람', '결혼']
keywords : ['소녀', '도사', '안녕', '이야기', '천재']
keywords : ['문자', '건의', '괴물', '기억', '달빛']
keywords : ['부쩍', '사이즈', '가족', '막내딸', '부부']
keywords : ['개그', '대신', '모스', '임시', '자칭']
keywords : ['이야기', '할머니', '가족', '그로', '변화']
keywords : ['집값', '한강', '시체', '여자', '유기한']
keywords : ['감정', '남자', '사건', '인간', '죽음']
keywords : ['갈등', '눈물', '사랑', '주인공', '타입']
keywords : ['불안감', '시험지', '전교', '정아']
keywords : ['괴담', '그날', '야자', '현실', '학교']
keywords : ['감당', '송곳', '기적', '부산', '감독']
keywords : ['공포', '단편', '대일', '연출', '이색']
keywords : ['시작', '악마', '의식', '통째', '소환']
keywords : ['가슴', '식당', '이야기', '할미']
keywords : ['마지막', '시간', '연애']
keywords : ['선녀', '이름', '순간', '순정', '남자']
keywords : ['국내', '네이버', '대표', '얼굴', '몰락']
keywords : ['다른', '선물', '신의', '장난']
keywords : ['갱생', '교사', '로맨스', '양호', '신부']
keyword

keywords : ['로맨스', '모든', '영화', '우민', '이혼']
keywords : ['강동구', '고등학생', '세계', '재벌']
keywords : ['동거', '사랑', '신작', '정서', '작가']
keywords : ['강아지', '인간', '자신', '주인']
keywords : ['어디']
keywords : ['다른', '만화', '제안', '편집']
keywords : ['로맨스', '물이', '저승', '학원', '판타지']
keywords : ['고개', '미시령', '지옥']
keywords : ['녀석', '마음', '모습', '상식', '소리']
keywords : ['오빠', '자리', '대신', '살해', '쌍둥이']
keywords : ['리지', '로만', '시리즈', '네이버웹툰', '듀얼']
keywords : ['로맨스', '숨기']
keywords : ['복희', '성장기', '식탐', '만두', '세상']
keywords : ['이탈', '인물', '남매', '예방', '이제']
keywords : ['감정', '비밀', '알파', '임무', '에이전트']
keywords : ['다른', '다사', '익숙', '주인공', '지금']
keywords : ['살인마', '쌍둥이', '운명', '정해진']
keywords : ['로맨스', '남자', '밀고', '여자', '저주']
keywords : ['군단', '미소녀', '본격', '소녀', '소년']
keywords : ['대표', '선수', '수영', '국가', '성장기']
keywords : ['갑자기', '동거', '인간', '투명']
keywords : ['부족', '생활', '시시', '청담', '적응']
keywords : ['로맨스', '대전', '수상', '수학', '청춘']
keywords : ['연화', '예전', '굴레', '수만', '실수']
keywords : ['액션', '전쟁', '거대', '로봇', '부산']
keywords

keywords : ['만화', '서울', '스카우트', '이야기', '인간']
keywords : ['위해', '학기', '개그', '단편', '당신']
keywords : ['달콤', '미남', '시작', '완벽', '운영']
keywords : ['계약', '모험', '사후세계', '시작', '소년']
keywords : ['시작', '학기', '경제', '고등학교', '국립']
keywords : ['시즌', '테리', '마야', '미스', '빙의']
keywords : ['선거', '성용', '아스', '만화', '스페이스']
keywords : ['모녀', '본격', '삼대', '엄마', '외할머니']
keywords : ['미국', '각양각색', '사람', '태권도', '프란시스']
keywords : ['조선', '수사', '승진', '추리', '한음']
keywords : ['만화', '동네', '법정', '본격', '생활']
keywords : ['마왕', '삼국', '삼국지', '이색', '전투기']
keywords : ['다른', '대화', '마음대로', '사람', '문자']
keywords : ['남녀', '그날', '이야기', '작가', '후유증']
keywords : ['일반', '중계', '규칙', '대결', '배틀']
keywords : ['드라마', '로맨틱', '지하철', '휴먼', '사람']
keywords : ['레드', '로맨스', '신작', '그린', '보이']
keywords : ['오빠', '남매', '리얼', '이야기', '진짜']
keywords : ['거대', '곤충', '대기', '상태', '습격']
keywords : ['능력', '생각', '조금']
keywords : ['게임', '남친', '대학', '수상작', '철벽']
keywords : ['누구', '대학', '마녀', '마지막', '만화']
keywords : ['그때', '로맨스', '마음', '사이', '이야기']
keywords : ['신작

keywords : ['가출', '거짓말', '격전', '만가', '대책']
keywords : ['서울', '희수', '남산', '로드무비', '만난']
keywords : ['개발', '게임', '발랄', '이야기', '좌충우돌']
keywords : ['중독', '학생', '대한민국', '선생', '스쿨']
keywords : ['말년', '방식', '이야기', '개그', '상상']
keywords : ['모든', '사고', '안대', '그때', '순간']
keywords : ['스포츠', '스페셜', '업데이트', '제공', '컨텐츠']
keywords : ['리아', '삼국지', '선율', '아프리카', '악기']
keywords : ['바로', '언제', '여행기', '지금', '고양이']
keywords : ['외모', '작가', '감동', '목욕', '상처']
keywords : ['사람', '무엇', '선의', '정체', '머리']
keywords : ['작가', '전설', '단편', '시즌', '이두']
keywords : ['관계', '빵점', '시작', '재활', '지령']
keywords : ['인기', '출동', '네이버', '릴레이', '마침내']
keywords : ['리메이크', '명작', '만화', '부천', '영상']
keywords : ['이의', '화랑', '꼭대기', '아파트', '옆집']
keywords : ['고등학교', '이야기', '친구', '톡톡']
keywords : ['불량', '건의', '먹잇감', '몸부림', '뭉치']
keywords : ['사랑', '열정', '커피']
keywords : ['러브', '레인저', '사연', '시즌', '어쩌']
keywords : ['네이버', '단편', '멸망', '이야기', '지구']
keywords : ['들판', '서로', '신화', '간직', '스페인']
keywords : ['오늘', '지금', '당신', '버스', '스릴러']
keywords : ['평화'

keywords : ['사람', '하루', '마음', '바로', '보이지']
keywords : ['보이', '여러', '오리가', '외로', '우정']
keywords : ['결혼', '생활', '이야기', '짐승']
keywords : ['가야', '누구', '마고', '삼국시대', '마음']
keywords : ['셀럽', '옥숙', '고등학교', '동창', '마스크']
keywords : ['수영', '도전', '백수', '예상', '위해']
keywords : ['매몰', '대학생', '발견', '사고', '상태']
keywords : ['인생', '자신', '꼬마', '정반대', '지금']
keywords : ['기지', '거기', '남작', '로맨스', '바트']
keywords : ['고품', '병맛', '아이', '이드']
keywords : ['갑자기', '고등학생', '기회', '여자애', '운명']
keywords : ['대상', '대한민국', '배우', '수상작', '인형']
keywords : ['인공', '지배', '거대', '기업', '문명']
keywords : ['피아노', '외국어', '갑부', '비관', '사고']
keywords : ['고양이', '세상', '천사', '소녀', '심장']
keywords : ['서연', '남녀', '생활', '서로', '선택']
keywords : ['언니', '먹이', '모든', '지배', '충격']
keywords : ['세상', '다른', '브랜든', '당신', '마지막']
keywords : ['살이', '시작', '업체', '잔혹', '직원']
keywords : ['리머', '스트', '국민', '삼류', '해결']
keywords : ['배달', '뱀파이어', '주소', '검정고시', '야식']
keywords : ['아람', '가장', '관심', '둘리', '마음']
keywords : ['인생', '일진', '괴롭힘', '시작', '왕따']
keywords : ['복수', 

keywords : ['경쟁', '광기', '단짝', '여고생', '전학생']
keywords : ['어둠', '이유', '현실', '대표', '네이버']
keywords : ['과연', '도시', '밀림', '영장', '동물']
keywords : ['대결', '마왕', '시댁', '남편', '목숨']
keywords : ['마왕', '산다', '대상', '대한민국', '두려움']
keywords : ['공포', '수상', '최대', '공모전', '수상작']
keywords : ['월영', '장군', '제국', '낯선', '만난']
keywords : ['만두', '이야기', '강아지', '거리', '동화']
keywords : ['마녀', '과거', '교단', '대사제', '롤랑']
keywords : ['결혼', '그날', '동생', '생일', '이면']
keywords : ['개발', '기술', '사람', '욕망', '인류']
keywords : ['바늘', '리지', '황후', '과거', '생각']
keywords : ['관인', '만인', '대한민국', '러리', '대리']
keywords : ['기사', '가족', '위해', '이레', '과거']
keywords : ['동료', '아침', '정이', '이별', '회사']
keywords : ['위해', '지수', '동화', '부터', '삽화가']
keywords : ['남편', '온갖', '패가망신', '관상', '도박']
keywords : ['소년', '친구', '좌절', '간다', '과정']
keywords : ['고통', '소년', '시작', '라이프', '전쟁']
keywords : ['공모전', '대상', '수상작', '최대', '거대']
keywords : ['미스터리', '신작', '하루아침', '호텔', '길거리']
keywords : ['사회', '윤리', '이야기', '인간', '도덕']
keywords : ['시선', '자신', '학교생활', '발견', '시체']
keyword

keywords : ['아연', '우울증', '감정', '사회', '세상']
keywords : ['망상', '소망', '남도', '성사', '소원']
keywords : ['대원', '로맨스', '뽀짝', '특공', '슈퍼']
keywords : ['동생', '문자', '수련', '군가', '그날']
keywords : ['연극', '직행', '계녀', '여자친구', '작가']
keywords : ['계약', '마주', '순간', '지하세계', '죽음']
keywords : ['편지', '동생', '동안', '달라', '사장']
keywords : ['마법사', '기숙', '독신', '문관', '사정']
keywords : ['쥬드', '그날', '바로', '손님', '이사']
keywords : ['최후', '이드', '그림', '간의', '미술']
keywords : ['하늘', '도깨비', '사기꾼', '이야기', '수장']
keywords : ['능력', '무엇', '범죄', '형사', '관계']
keywords : ['모태', '취해', '고백', '규리', '기억']
keywords : ['손녀', '당신', '머리', '복수', '세탁']
keywords : ['연애', '연하', '막내', '시작', '신입사원']
keywords : ['영은', '대회', '비밀', '도둑', '수영']
keywords : ['변신', '얼굴', '생활', '스릴러', '계약']
keywords : ['가야', '모범생', '모두', '시절', '여왕']
keywords : ['한지', '막내', '아들', '가의', '대한민국']
keywords : ['유별', '샤인', '달리기', '관심', '드라마']
keywords : ['호랑이', '결말', '로맨스', '시대', '녀석']


In [6]:
naver1 = pd.read_csv(path + '네이버_추천시스템.csv' , encoding='cp949')

In [7]:
naver1['keywords'] = key_wordlist

In [8]:
naver1.to_csv('네이버_웹툰_최최종.csv')

## Daum

In [9]:
web_combined = daum

In [10]:
key_wordlist2 = []
import warnings
warnings.filterwarnings('ignore')

for i in range(0, len(web_combined['story'])):
    try :
        
        text = web_combined['story'][i]
        textrank = TextRank(text)
        print('keywords :', textrank.keywords())
        key_wordlist2.append(textrank.keywords())
        
    except:
        key_wordlist2.append("none")

keywords : ['좌충우돌', '카페', '다른', '동업', '드라마']
keywords : ['덕질', '마음', '아이', '위해']
keywords : ['선수', '방송', '요한', '졸업', '지망']
keywords : ['악마', '사회', '위협', '적응', '존재']
keywords : ['다시', '사내', '수저', '천하', '제일']
keywords : ['무엇', '정체', '자꾸만', '의사', '주변']
keywords : ['노예', '명월', '전국시대', '기생', '무림']
keywords : ['세상', '작가', '조경', '최선']
keywords : ['고전', '단영', '복수', '성장', '오누이']
keywords : ['식당', '백로', '신들', '요리', '음식']
keywords : ['구역', '남자', '사랑', '여자', '수거']
keywords : ['도전', '리스', '시험', '주인공', '전투']
keywords : ['라이프', '조선시대', '집사', '나무꾼', '동물']
keywords : ['소녀', '시스트', '엑소', '이야기']
keywords : ['로맨스', '발레리', '철벽', '환생']
keywords : ['댄스', '거머', '우승', '위드', '파트너']
keywords : ['재능', '합작', '그린', '미술', '스타']
keywords : ['감염', '바이러스', '소가', '존재', '수용']
keywords : ['세상', '신인', '어차피', '제정']
keywords : ['귀신', '숙경', '이야기', '학교', '존재']
keywords : ['윤정', '이야기', '학교']
keywords : ['녀석', '시작', '아빠', '엄마']
keywords : ['공개', '낚는', '라면', '청산', '생존']
keywords : ['도움', '귀신', '소년', '의문', '학교']
keywords : ['데트', 

keywords : ['남북', '상치', '위기', '인의', '위해']
keywords : ['공격', '생명체', '소녀', '크리', '이야기']
keywords : ['문제', '세상', '이건', '혁명']
keywords : ['개장', '동물', '동물원']
keywords : ['가슴', '로맨스', '시대극', '진실', '애증']
keywords : ['눈빛', '아이']
keywords : ['유령', '천재', '귀신', '라며', '미녀']
keywords : ['뷰티', '크리', '관심', '그림', '로맨스']
keywords : ['기회', '남녀', '눈앞', '모든', '때마침']
keywords : ['어린이', '예술가', '욕망', '하나']
keywords : ['오늘', '혹시']
keywords : ['시대', '몸부림', '시작', '욕망', '짐승']
keywords : ['끼리', '밌으', '세상', '끼리끼리', '츠기']
keywords : ['세계', '승은', '후궁']
keywords : ['남녀', '사고', '안개', '탈출']
keywords : ['가장', '다른', '자신', '존재', '장소']
keywords : ['궁합', '다른', '성격', '연상', '연하남']
keywords : ['남자', '십년', '외출']
keywords : ['공인', '데뷔', '사건', '일대', '자타']
keywords : ['액션', '코믹', '간의', '기억', '드일행']
keywords : ['껍데기', '도하', '성형', '의사', '외과']
keywords : ['매몰', '발신', '전화', '표시', '제한']
keywords : ['등골', '기어', '벌레', '오싹', '찰나']
keywords : ['남자', '라이프', '서울', '여자', '시골']
keywords : ['대판', '대하', '심층', '논리', '만화']
keywords : ['규칙', '노인', 

keywords : ['고양이', '다른', '상황', '인간', '생각']
keywords : ['담임', '마음', '이상']
keywords : ['다른', '스타일', '전혀', '지금']
keywords : ['사랑', '제대로', '지금']
keywords : ['순간', '심장', '일상', '기억']
keywords : ['괴물', '남자', '아이', '아이돌']
keywords : ['농촌', '대왕', '파워', '향연']
keywords : ['도시', '무조건', '아웃', '화이트', '전체']
keywords : ['본적', '구조', '산악', '이드', '이제껏']
keywords : ['다른', '동물', '메트로놈', '원진', '반려']
keywords : ['화성']
keywords : ['개인', '나태', '다큐', '서울대']
keywords : ['세기', '홈즈', '가상', '각종', '국가']
keywords : ['은혜', '형제', '놀부', '부와', '사이']
keywords : ['가지', '단편', '버스', '사람']
keywords : ['동구', '네온', '부부', '자동차', '캐러멜']
keywords : ['드비', '이야기', '일상', '작가']
keywords : ['명예', '수만', '지위']
keywords : ['상실', '오감', '강의', '기억', '누구']
keywords : ['교향악단', '시립', '음악', '이야기']
keywords : ['다운', '드라마', '미스', '변호사', '법정']
keywords : ['바로', '순금']
keywords : ['사무소', '사건', '사설', '이야기', '탐정']
keywords : ['기록', '기네스', '기상', '외한', '이야기']
keywords : ['공모전', '그날', '다음', '상작', '최우수']
keywords : ['교실', '여왕', '정말', '지배']
keywords : ['고양이

keywords : ['무시', '이야기', '학교']
keywords : ['안고', '입학', '군대', '기분', '마음']
keywords : ['사랑', '시공', '초월']
keywords : ['냄새', '빵집', '옥탑', '회사', '이야기']
keywords : ['기차역', '로맨스', '전화위복', '취업']
keywords : ['가입', '미션', '발표', '창단', '선물']
keywords : ['대결', '막걸리', '신명', '최고']
keywords : ['사랑', '생각', '오토바이']
keywords : ['미식', '광동', '광저우', '동아리', '렛츠고']
keywords : ['거리', '스트릿', '파이트']
keywords : ['남녀', '서로', '이야기']
keywords : ['만남', '사람']
keywords : ['가득', '동물', '소리']
keywords : ['절대', '혁명', '계속', '귀환', '악의']
keywords : ['고통', '기생', '김삿갓', '영월', '조선']
keywords : ['달리', '준수', '드라마', '성장', '푸른']
keywords : ['드립', '끝판', '순도', '유희', '키몽']
keywords : ['묵직', '사생활', '웹툰작가']
keywords : ['계세', '당신', '대해', '얼마나', '사람']
keywords : ['동물', '각양각색', '드라마', '버스', '야생']
keywords : ['기관', '대립', '욕망', '권력', '사회']
keywords : ['액션', '의학', '국제', '대한민국', '무도']
keywords : ['세상', '악마', '엘리', '일대기', '영웅']
keywords : ['낭만', '귀신', '대책', '처녀', '취준생']
keywords : ['스릴러', '위기', '이야기', '전쟁', '첩보']
keywords : ['랜드', '스코', '로드', '버디'

In [11]:
daum1 = pd.read_csv(path + 'daum_final.csv', encoding = 'cp949')

daum1['keywords'] = key_wordlist2

In [12]:
daum1.to_csv('다음_웹툰_최최종.csv')